In [3]:
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt
from sklearn.ensemble import RandomForestRegressor, RandomForestClassifier
from sklearn.model_selection import RandomizedSearchCV
from sklearn.random_projection import GaussianRandomProjection
from sklearn.model_selection import train_test_split
from sklearn.decomposition import PCA
from sklearn import metrics
import shap
from sklearn.manifold import TSNE
import matplotlib.pyplot as plt
from tqdm import tqdm
import os
from utils import *
from skgarden import RandomForestQuantileRegressor

%matplotlib notebook
%pylab

D:\Programs\Anaconda3\lib\site-packages\sklearn\linear_model\least_angle.py:30: DeprecationWarning: `np.float` is a deprecated alias for the builtin `float`. To silence this warning, use `float` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.float64` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  method='lar', copy_X=True, eps=np.finfo(np.float).eps,
D:\Programs\Anaconda3\lib\site-packages\sklearn\linear_model\least_angle.py:167: DeprecationWarning: `np.float` is a deprecated alias for the builtin `float`. To silence this warning, use `float` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.float64` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  method='lar', copy_X=True, eps=n

Using matplotlib backend: nbAgg
Populating the interactive namespace from numpy and matplotlib


In [8]:
datau = pd.read_csv('./Rate=UNOS/Batch Simulation Sun Jan 10 23-35-55 EST 2021/data.csv')

In [9]:
_, _, Xmu, Yiu, _ = data_to_input(datau)

In [10]:
rfu = RandomForestQuantileRegressor(
    random_state=0, min_samples_split=10, n_estimators=1000)

In [11]:
rfu.fit(Xmu, Yiu)

RandomForestQuantileRegressor(bootstrap=True, criterion='mse', max_depth=None,
                              max_features='auto', max_leaf_nodes=None,
                              min_samples_leaf=1, min_samples_split=10,
                              min_weight_fraction_leaf=0.0, n_estimators=1000,
                              n_jobs=1, oob_score=False, random_state=0,
                              verbose=0, warm_start=False)

In [12]:
s1 = pd.read_csv('./Rate=UNOS/sample1.csv')

In [13]:
Xms, Yis = data_to_iterations(s1)

In [48]:
def iou(pl, ph, Y, p):
    tl = np.percentile(Y, (100-p)/2)
    th = np.percentile(Y, 100-(100-p)/2)
    
    ll = min(pl, tl)
    lh = max(pl, tl)
    
    hl = min(ph, th)
    hh = max(ph, th)
    
    if lh > hl:
        return 0
    return (hl - lh)/(hh - ll)

In [39]:
def ious(path, path2, p):
    data = pd.read_csv(path+'/data.csv')
    _, _, X, Y, _ = data_to_input(data)
    rf = RandomForestQuantileRegressor(
    random_state=1, min_samples_split=10, n_estimators=1000)
    rf.fit(X,Y)
    
    ious = []
    for i in tqdm(range(1,301)):
        sample = pd.read_csv(path2 + "/sample"+str(i)+".csv")
        Xt, Yt = data_to_iterations(sample)
        if Xt.shape[0] > 0:
            ious.append(iou(rf.predict(Xt, (100-p)/2), rf.predict(Xt, 100-(100-p)/2), Yt, p))
    
    return np.array(ious)

In [3]:
unos = ious('./Rate=UNOS/Batch Simulation Sun Jan 10 23-35-55 EST 2021', './Rate=UNOS', 95)

100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 300/300 [01:35<00:00,  3.14it/s]


In [62]:
rate3 = ious('./Rate=3/Batch Simulation Tue May 18 22-39-09 EDT 2021', './Rate=3', 95)

100%|████████████████████████████████████████████████████████████████████████████████| 300/300 [02:14<00:00,  2.23it/s]


In [63]:
rate2 = ious('./Rate=2/Batch Simulation Sun Mar 14 01-58-15 EST 2021', './Rate=2', 95)

100%|████████████████████████████████████████████████████████████████████████████████| 300/300 [02:13<00:00,  2.25it/s]


In [72]:
rate1 = ious('./Rate=1/Batch Simulation Mon May 17 00-21-59 EDT 2021', './Rate=1', 95)

100%|████████████████████████████████████████████████████████████████████████████████| 300/300 [02:08<00:00,  2.33it/s]


In [65]:
rate1l = ious('./Rate=1, Long/Long', './Rate=1, Long', 95)

100%|████████████████████████████████████████████████████████████████████████████████| 300/300 [02:31<00:00,  1.99it/s]


In [66]:
print(unos.mean())

0.5027539809602798


In [67]:
print(rate3.mean())

0.38635311102163683


In [68]:
print(rate2.mean())

0.3360101614242867


In [73]:
print(rate1.mean())

0.4507145398860816


In [70]:
print(rate1l.mean())

0.6531681749862589


In [4]:
print(np.percentile(unos,50))

0.4408998565430597


In [29]:
def fedious(q,p):
    data = pd.read_csv('./Federated'+str(q)+'/1/data.csv')
    _, _, X, Y, _ = data_to_input(data)
    for i in range(2,6):
        datai = pd.read_csv('./Federated'+str(q)+'/'+str(i)+'/data.csv')
        _, _, Xi, Yi, _ = data_to_input(datai)
        X = pd.concat([X,Xi])
        Y = pd.concat([Y,Yi])
    rf = RandomForestQuantileRegressor(
    random_state=1, min_samples_split=10, n_estimators=1000)
    rf.fit(X,Y)
    
    ious = []
    for i in tqdm(range(1,301)):
        sample = pd.read_csv("./Federated"+str(q)+"/sample"+str(i)+".csv")
        Xt, Yt = data_to_iterations(sample)
        if Xt.shape[0] > 0:
            ious.append(iou(rf.predict(Xt, (100-p)/2), rf.predict(Xt, 100-(100-p)/2), Yt, p))
    
    return np.array(ious)

In [8]:
fedrate2 = fedious(95)

100%|████████████████████████████████████████████████████████████████████████████████| 300/300 [01:31<00:00,  3.27it/s]


In [9]:
fedrate2.mean()

0.3732441070121499

In [12]:
np.percentile(fedrate2,50)

0.32699559314188675

In [30]:
fedrate1=fedious(1,95)

100%|████████████████████████████████████████████████████████████████████████████████| 300/300 [01:30<00:00,  3.33it/s]
Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.


In [32]:
fedrate1.mean()

array([0.23155905])

In [33]:
fedrate3=fedious(3,95)

100%|████████████████████████████████████████████████████████████████████████████████| 300/300 [01:39<00:00,  3.00it/s]


In [34]:
fedrate3.mean()

0.421210770805397

In [35]:
fedrateunos=fedious('UNOS',95)

100%|████████████████████████████████████████████████████████████████████████████████| 300/300 [01:36<00:00,  3.10it/s]


In [38]:
fedrateunos.mean()

0.44850027973884715

In [37]:
np.percentile(fedrateunos,50)

0.37911331327824394

In [40]:
rate2l = ious('./Rate=2, Long/Long', './Rate=2, Long', 95)

100%|████████████████████████████████████████████████████████████████████████████████| 300/300 [01:31<00:00,  3.26it/s]


In [41]:
rate2l.mean()

0.6200849322702476

In [42]:
fedrate2l = fedious('2l',95)

100%|████████████████████████████████████████████████████████████████████████████████| 300/300 [01:39<00:00,  3.00it/s]


In [43]:
fedrate2l.mean()

0.7757536987737705

In [44]:
rate1l2 = ious('./Rate=1, Long2/Long', './Rate=1, Long2', 95)

100%|████████████████████████████████████████████████████████████████████████████████| 300/300 [01:33<00:00,  3.21it/s]


In [45]:
rate1l2.mean()

0.6439813197082409

In [51]:
fedrate1l = fedious('1l',95)

100%|████████████████████████████████████████████████████████████████████████████████| 300/300 [01:34<00:00,  3.18it/s]


In [52]:
fedrate1l.mean()

0.5908324938575806